In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import xgboost
import shap
import warnings
from scipy.stats import chi2_contingency
import re
import torch
import random
from sklearn.decomposition import PCA, KernelPCA
import torch.nn as nn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from lifelines import KaplanMeierFitter
from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from lifelines.statistics import multivariate_logrank_test
from sklearn.decomposition import PCA
from scipy.stats import kruskal
import math
import os
import snf
from sklearn.cluster import spectral_clustering
from sklearn.metrics import silhouette_score, silhouette_samples
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import rpy2.robjects as robjects
from rpy2.robjects import r,pandas2ri
from rpy2.robjects.packages import importr
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, accuracy_score
from sklearn.metrics.cluster import contingency_matrix
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sklearn.pipeline import make_pipeline
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.preprocessing import StandardScaler
from lpproj import LocalityPreservingProjection 
from sklearn.manifold import TSNE
def spectral_clustering_nSNN(A, k):
    data_size = A.shape[0]
    B = np.zeros((data_size, data_size))  # Create a similarity matrix which is fully connected

    # Compute similarity matrix using Gaussian function
    for i in range(data_size):
        for j in range(data_size):
            B[i, j] = np.exp(-np.sum((A[i, :] - A[j, :]) ** 2) / (2 * 1.5 ** 2))
            B[j, i] = B[i, j]

    temp = np.array([sorted(row, reverse=True) for row in B])  # Recording the distance in B from big to small
    I = np.argsort(-B, axis=1)  # I is the corresponding id

    for i in range(k, data_size):
        temp[:, i] = 0

    # E is the similarity matrix of the k nearest neighbors
    E = np.zeros((data_size, data_size))
    for i in range(data_size):
        for j in range(k):
            E[i, I[i, j]] = temp[i, j]

    E[np.where(E != 0)] = 1  # Replace nonzero sparse matrix elements with ones
    G = np.copy(E)

    W = np.zeros((data_size, data_size))  # W is the similarity matrix of the shared nearest neighbors

    for i in range(data_size):
        for j in range(i + 1, data_size):
            diff = np.sum(np.abs(G[i, :] - G[j, :])) / 2
            W[i, j] = k - diff
            if G[i, j] != 0 and G[j, i] != 0:
                W[i, j] += 1
            W[i, j] /= k
            W[j, i] = W[i, j]

    # Normalized Laplacian Matrix
    degrees = np.sum(W, axis=1)
    sqrt_degrees = np.sqrt(degrees)
    normalized_laplacian = np.diag(1.0 / sqrt_degrees) @ (np.diag(degrees) - W) @ np.diag(1.0 / sqrt_degrees)

    # Eigen Decomposition
    eigenvalues, eigenvectors = np.linalg.eig(normalized_laplacian)
    sorted_indices = np.argsort(eigenvalues)
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    eigengaps = np.diff(np.sort(eigenvalues))
    max_gap_index = np.argmax(eigengaps)
    num_clusters = max(2, max_gap_index + 1 if max_gap_index < 6 else 6)
    num_clusters = 3
    # K-means Clustering on Eigenvectors
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    clusters = kmeans.fit_predict(np.real(sorted_eigenvectors[:, :num_clusters]))

    return clusters, W, num_clusters
    

In [ ]:
type = "ESCA"
gene = pd.read_csv("/Users/shi/Documents/OmicsDatapreprocessed/" + type + "_RNAseq.csv", index_col=0)
methy = pd.read_csv("/Users/shi/Documents/OmicsDatapreprocessed/" + type + "_protein.csv", index_col=0)
mirna = pd.read_csv("/Users/shi/Documents/OmicsDatapreprocessed/" + type + "_miRNAseq.csv", index_col=0)
pathology = pd.read_csv("/Users/shi/Documents/TCGA_Reports.csv", index_col=0, )
pathology.index = pathology.index.str.extract(r'^(TCGA-\w+-\w+)')[0].str.replace('-', '.')
common_index = gene.index.intersection(pathology.index) 
filtered_pathology = pathology.loc[common_index]        
gene = gene.loc[common_index]       
methy = methy.loc[common_index]       
mirna = mirna.loc[common_index]       
filtered_pathology.to_csv('ESCA_patho.csv')

clinical = pd.read_csv("/Users/shi/Documents/OmicsDatapreprocessed/" + type +"_clinical.csv")
clinical.set_index(clinical.columns[0], inplace=True)
clinical = clinical.loc[common_index]
clinical['Time'] = clinical['Time'].fillna(clinical['Time'].median())
clinical['Death'] = clinical['Death'].fillna(clinical['Death'].median())
clinical['age'] = clinical['age'].fillna(clinical['age'].median())
clinicalnew = clinical.reset_index()

In [ ]:
gene.to_csv("/Users/shi/Documents/ori/ESCA_RNAseq_ori.csv")
methy.to_csv("/Users/shi/Documents/ori/ESCA_protein_ori.csv")
mirna.to_csv("/Users/shi/Documents/ori/ESCA_miRNAseq_ori.csv")

In [ ]:
embed = pd.read_csv("/Users/shi/Documents/OmicsDatapreprocessed/ESCA_embeddings.csv", index_col=0).iloc[:, 1:]
print(gene.shape)
print(methy.shape)
print(mirna.shape)
print(embed.shape)

In [ ]:
gene_fused = pd.read_csv("/Users/shi/Documents/fused/ESCA_RNAseq_fused.csv", index_col=0)
methy_fused = pd.read_csv("/Users/shi/Documents/fused/ESCA_protein_fused.csv", index_col=0)
mirna_fused = pd.read_csv("/Users/shi/Documents/fused/ESCA_miRNAseq_fused.csv", index_col=0)
print(gene_fused.shape)
print(methy_fused.shape)
print(mirna_fused.shape)
completed_data = np.concatenate([gene_fused, methy_fused, mirna_fused], axis=1)
final_representation_np, _, _ = np.linalg.svd(completed_data)

In [ ]:
pandas2ri.activate()
umap = importr('umap')
cancersubtypes = importr('CancerSubtypes')

dim = 10  
k = 20  

U1 = final_representation_np[:, :dim]

clusters, similar, clusternumber = spectral_clustering_nSNN(U1, k)

T = clinical['Time']
E = clinical['Death']
results = multivariate_logrank_test(T, clusters, E)
p_val_avg = results.p_value

sil = cancersubtypes.silhouette_SimilarityMatrix(clusters, pandas2ri.DataFrame(pd.DataFrame(similar)))
silhouette_width = np.mean(sil[:, 2])

print('Cluster number:', clusternumber)
print(f'dim={dim}, k={k}: p-value = {p_val_avg}, -log10(p-value) ={-math.log10(p_val_avg)} ')

clinical['pathologic_S_numeric'], _ = pd.factorize(clinical['pathologic_stage'])
chi2_S, p_value_S, _, _ = chi2_contingency(pd.crosstab(clinical['pathologic_S_numeric'], clusters))
clinical['pathologic_T_numeric'], _ = pd.factorize(clinical['pathologic_T'])
chi2_T, p_value_T, _, _ = chi2_contingency(pd.crosstab(clinical['pathologic_T_numeric'], clusters))
clinical['pathologic_M_numeric'], _ = pd.factorize(clinical['pathologic_M'])
chi2_M, p_value_M, _, _ = chi2_contingency(pd.crosstab(clinical['pathologic_M_numeric'], clusters))
clinical['pathologic_N_numeric'], _ = pd.factorize(clinical['pathologic_N'])
chi2_N, p_value_N, _, _ = chi2_contingency(pd.crosstab(clinical['pathologic_N_numeric'], clusters))
clinical['gender_numeric'], _ = pd.factorize(clinical['gender'])
chi2_G, p_value_G, _, _ = chi2_contingency(pd.crosstab(clinical['gender_numeric'], clusters))
statistic, p_value_a = kruskal(clusters, clinical['age'])
significant_count = sum(p < 0.05 for p in [p_value_S, p_value_T, p_value_M, p_value_N, p_value_G, p_value_a])
print("Significant:", significant_count)
print("silhouette_width:", silhouette_width)
